Zahara Haghshenas


Narjes Sheikhpour Shirazi

##data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [ ]:
%ls

drive/  flagged/  sample_data/


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale, StandardScaler, normalize
from yellowbrick.cluster import KElbowVisualizer
from sklearn.manifold import TSNE
import seaborn as sns
import random

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/kaggle/ratings.csv')
df2=pd.read_csv('/content/drive/MyDrive/kaggle/movies_metadata.csv')
df3=pd.read_csv('/content/drive/MyDrive/kaggle/ratings_small.csv')

<ipython-input-12-e42a4e8736e1>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv('/content/drive/MyDrive/kaggle/movies_metadata.csv')


In [ ]:
df1


userId  movieId  rating   timestamp
0              1      110     1.0  1425941529
1              1      147     4.5  1425942435
2              1      858     5.0  1425941523
3              1     1221     5.0  1425941546
4              1     1246     5.0  1425941556
...          ...      ...     ...         ...
26024284  270896    58559     5.0  1257031564
26024285  270896    60069     5.0  1257032032
26024286  270896    63082     4.5  1257031764
26024287  270896    64957     4.5  1257033990
26024288  270896    71878     2.0  1257031858

[26024289 rows x 4 columns]

In [ ]:
df2

adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                                  genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                         [{'id': 35, 'name': 'Comedy'}]   
...                                                  ...   
45461  [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...   
45462                      [{'id': 18, 'name': 'Drama'}]   
45463  [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...   
45464                                                 []   
45465                                                 []   

                                   homepage      id    imdb_id  \
0      http://toystory.disney.com/toy-story     862  tt0114709   
1                                       NaN    8844  tt0113497   
2                                       NaN   15602  tt0113228   
3                                       NaN   31357  tt0114885   
4                                       NaN   11862  tt0113041   
...                                     ...     ...        ...   
45461  http://www.imdb.com/title/tt6209470/  439050  tt6209470   
45462                                   NaN  111109  tt2028550   
45463                                   NaN   67758  tt0303758   
45464                                   NaN  227506  tt0008536   
45465                                   NaN  461257  tt6980792   

      original_language               original_title  \
0                    en                    Toy Story   
1                    en                      Jumanji   
2                    en             Grumpier Old Men   
3                    en            Waiting to Exhale   
4                    en  Father of the Bride Part II   
...                 ...                          ...   
45461                fa                      رگ خواب   
45462                tl          Siglo ng Pagluluwal   
45463                en                     Betrayal   
45464                en          Satana likuyushchiy   
45465                en                     Queerama   

                                                overview  ... release_date  \
0      Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1      When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2      A family wedding reignites the ancient feud be...  ...   1995-12-22   
3      Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4      Just when George Banks has recovered from his ...  ...   1995-02-10   
...                                                  ...  ...          ...   
45461        Rising and falling between a man and woman.  ...          NaN   
45462  An artist struggles to finish his work while a...  ...   2011-11-17   
45463  When one of her hits goes wrong, a professiona...  ...   2003-08-01   
45464  In a small town live two brothers, one a minis...  ...   1917-10-21   
45465  50 years after decriminalisati

In [ ]:
df2.shape

(45466, 24)

In [ ]:
df2.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

##Cluster-Based: these methods are a popular approach in recommender systems that
aim to group users or items into clusters based on their similarities. These clusters can
then be used to make recommendations by leveraging the preferences or characteristics
of other users or items within the same cluster.
Recommendations can be generated by considering the clusters to which the input
movies belong. Alternatively, movies from different clusters can be recommended to
introduce new content to the user. Another option is to recommend movies that are most
similar to the input movies or those located at the center of the clusters. Additionally,
creative approaches can be used to generate recommendations. Extra scores may be
granted for creative approaches

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity

# Load the movies dataset
movies = pd.read_csv("/content/drive/MyDrive/kaggle/movies_metadata.csv", low_memory=False)

# Preprocess the data
# Drop irrelevant columns
movies = movies.drop(["adult", "belongs_to_collection", "homepage", "imdb_id", "original_title", "overview", "poster_path", "status", "tagline", "video"], axis=1)

# Convert budget and revenue to numeric
movies["budget"] = pd.to_numeric(movies["budget"], errors="coerce")
movies["revenue"] = pd.to_numeric(movies["revenue"], errors="coerce")

# Convert release_date to datetime
movies["release_date"] = pd.to_datetime(movies["release_date"], errors="coerce")

# Extract year and month from release_date
movies["year"] = movies["release_date"].dt.year
movies["month"] = movies["release_date"].dt.month

# Drop release_date column
movies = movies.drop("release_date", axis=1)

# Convert genres, spoken_languages, production_countries and production_companies to lists of names
def get_names(x):
    # Convert stringified JSON to list of dictionaries
    # Use a try-except block to catch any exceptions
    try:
        x = eval(x)
        # Extract the name attribute from each dictionary
        names = [d["name"] for d in x]
    except:
        # Return an empty list if evaluation fails
        names = []
    # Return the list of names
    return names

movies["genres"] = movies["genres"].apply(get_names)
movies["spoken_languages"] = movies["spoken_languages"].apply(get_names)
movies["production_countries"] = movies["production_countries"].apply(get_names)
movies["production_companies"] = movies["production_companies"].apply(get_names)

# Fill missing values with zeros
movies.fillna({"budget": 0, "revenue": 0, "popularity": 0, "runtime": 0, "vote_count": 0, "vote_average": 0}, inplace=True)

# Create a feature matrix by one-hot encoding the categorical features
# Concatenate the lists of names for each movie
movies["all_names"] = movies["genres"] + movies["spoken_languages"] + movies["production_countries"] + movies["production_companies"]

# Get the unique names across all movies
all_names = set()
for names in movies["all_names"]:
    all_names.update(names)

# Create a dictionary to map each name to an index
name_to_index = {name: i for i, name in enumerate(all_names)}

# Create an empty matrix of zeros with shape (number of movies, number of names)
feature_matrix = np.zeros((len(movies), len(all_names)))

# Loop over each movie and its names
for i, names in enumerate(movies["all_names"]):
    # Loop over each name and get its index
    for name in names:
        j = name_to_index[name]
        # Set the corresponding element in the feature matrix to 1
        feature_matrix[i, j] = 1

# Cluster the movies using Mini-Batch KMeans algorithm
# Choose the number of clusters (can be tuned using some criteria)
k = 10

# Create a MiniBatchKMeans object with k clusters and random state for reproducibility
kmeans = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=100)

# Fit the feature matrix to the KMeans object
kmeans.fit(feature_matrix)

# Get the cluster labels for each movie
labels = kmeans.labels_

# Add the cluster labels to the movies dataframe
movies["cluster"] = labels

# Define a function to get movie recommendations based on cluster-based approach
def get_recommendations(input1,input2,input3):
    # Initialize an empty list to store the recommendations
    recommendations = []
    input_movies=[input1,input2,input3]

    # Loop over each input movie
    for input_movie in input_movies:
        # Find the movie in the movies dataframe
        movie = movies[movies["title"] == input_movie]

        # If the movie is not found, print a message and continue
        if movie.empty:
            print(f"{input_movie} is not in the dataset.")
            continue

        # Get the cluster label of the movie
        cluster = movie["cluster"].iloc[0]

        # Get all the movies in the same cluster as the input movie
        similar_movies = movies[movies["cluster"] == cluster]

        # Sort the similar movies by popularity in descending order
        similar_movies = similar_movies.sort_values(by="popularity", ascending=False)

        # Get the top 10 most popular movies in the same cluster
        top_movies = similar_movies["title"].head(10)

        # Add the top movies to the recommendations list
        recommendations.extend(top_movies)

    # Return the recommendations list
    return recommendations

# Get some input movies from the user
#input_movies = ["The Matrix", "The Godfather", "Titanic"]

# Get the recommendations based on the input movies
recommendations = get_recommendations("The Matrix", "The Godfather", "Titanic")

# Print the recommendations
print("Here are some movies you might like:")
for movie in recommendations:
    print(movie)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Here are some movies you might like:
Dinosaur
Jarhead
The Last Kiss
Black Dawn
Black Sea
Hollywood Ending
The Haunting of Molly Hartley
Charlie Wilson's War
Down and Out in Beverly Hills
The Next Karate Kid
Dinosaur
Jarhead
The Last Kiss
Black Dawn
Black Sea
Hollywood Ending
The Haunting of Molly Hartley
Charlie Wilson's War
Down and Out in Beverly Hills
The Next Karate Kid
Blade Runner
Dream House
Uncommon Valor
The Lords of Salem
Critters 2
Brainstorm
El Mariachi
Original Sin
Arena
Foreign Correspondent


In [ ]:
input_movies = ["The Matrix", "The Godfather", "Titanic"]

In [ ]:
!pip install gradio

In [ ]:
import numpy as np
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")

    input1 = gr.Textbox(placeholder='first film name...')

    input2 = gr.Textbox(placeholder='second film name...')

    input3 = gr.Textbox(placeholder='third film name...')

    text_output=gr.Textbox( )


    text_button = gr.Button("Submit")
    text_button.click( get_recommendations, inputs=[input1,input2,input3], outputs=text_output)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(movie1, movie2):
    # Get the feature vectors for the two movies
    vector1 = feature_matrix[movies[movies["title"] == movie1].index]
    vector2 = feature_matrix[movies[movies["title"] == movie2].index]

    # Calculate the cosine similarity between the two feature vectors
    similarity = cosine_similarity(vector1, vector2)[0][0]

    return similarity


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans

# Load the movies dataset
movies = pd.read_csv("/content/drive/MyDrive/kaggle/movies_metadata.csv")

# Preprocess the data
# Drop irrelevant columns
movies = movies.drop(["adult", "belongs_to_collection", "homepage", "imdb_id", "original_title", "overview", "poster_path", "status", "tagline", "video"], axis=1)

# Convert budget and revenue to numeric
movies["budget"] = pd.to_numeric(movies["budget"], errors="coerce")
movies["revenue"] = pd.to_numeric(movies["revenue"], errors="coerce")

# Convert release_date to datetime
movies["release_date"] = pd.to_datetime(movies["release_date"], errors="coerce")

# Extract year and month from release_date
movies["year"] = movies["release_date"].dt.year
movies["month"] = movies["release_date"].dt.month

# Drop release_date column
movies = movies.drop("release_date", axis=1)

# Convert genres, spoken_languages, production_countries, and production_companies to lists of names
def get_names(x):
    try:
        x = eval(x)
        names = [d["name"] for d in x]
    except:
        names = []
    return names

movies["genres"] = movies["genres"].apply(get_names)
movies["spoken_languages"] = movies["spoken_languages"].apply(get_names)
movies["production_countries"] = movies["production_countries"].apply(get_names)
movies["production_companies"] = movies["production_companies"].apply(get_names)

# Fill missing values with zeros
movies = movies.fillna(0)

# Forward fill missing values for selected columns
columns_to_fill_forward = ["budget", "revenue", "popularity", "runtime", "vote_count", "vote_average"]
movies[columns_to_fill_forward] = movies[columns_to_fill_forward].fillna(method='ffill')

# Create a feature matrix by one-hot encoding the categorical features
movies["all_names"] = movies["genres"] + movies["spoken_languages"] + movies["production_countries"] + movies["production_companies"]

# Get the unique names across all movies
all_names = set()
for names in movies["all_names"]:
    all_names.update(names)

# Create a dictionary to map each name to an index
name_to_index = {name: i for i, name in enumerate(all_names)}

# Create an empty matrix of zeros with shape (number of movies, number of names)
feature_matrix = np.zeros((len(movies), len(all_names)))

# Loop over each movie and its names
for i, names in enumerate(movies["all_names"]):
    # Loop over each name and get its index
    for name in names:
        j = name_to_index[name]
        # Set the corresponding element in the feature matrix to 1
        feature_matrix[i, j] = 1

# Cluster the movies using MiniBatchKMeans algorithm
k = 10
kmeans = MiniBatchKMeans(n_clusters=k, random_state=42)
kmeans.fit(feature_matrix)

# Get the cluster labels for each movie
labels = kmeans.labels_

# Add the cluster labels to the movies dataframe
movies["cluster"] = labels

# Define a function to get movie recommendations based on item-based collaborative filtering
def get_item_based_recommendations(input1,input2,input3):
    # Initialize an empty dictionary to store the similarity scores
    similarity_scores = {}
    input_movies=[input1,input2,input3]
    # Loop over each input movie
    for input_movie in input_movies:
        # Find the movie in the movies dataframe
        movie = movies[movies["title"] == input_movie]

        # If the movie is not found, print a message and continue
        if movie.empty:
            print(f"{input_movie} is not in the dataset.")
            continue

        # Get the cluster label of the movie
        cluster = movie["cluster"].iloc[0]

        # Get all the movies in the same cluster as the input movie
        similar_movies = movies[movies["cluster"] == cluster]

        # Calculate the similarity score between the input movie and each similar movie
        for index, row in similar_movies.iterrows():
            if row["title"] not in input_movies:
                # Calculate the similarity score based on some criteria
                similarity_score = calculate_similarity(input_movie, row["title"])

                # Add the similarity score to the dictionary
                if row["title"] in similarity_scores:
                    similarity_scores[row["title"]] += similarity_score
                else:
                    similarity_scores[row["title"]] = similarity_score

    # Sort the movies based on the similarity scores in descending order
    sorted_movies = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)

    # Get the top 10 most similar movies
    top_movies = [movie[0] for movie in sorted_movies[:10]]

    # Return the recommendations list
    return top_movies

# Get the recommendations based on the input movies using the item-based collaborative filtering approach
recommendations = get_item_based_recommendations("The Matrix", "The Godfather", "Titanic")

# Print the recommendations
print("\nHere are some movies you might like (item-based collaborative filtering approach):",recommendations)
#for movie in recommendations:
 #   print(movie)



<ipython-input-13-d8f1f31fea70>:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("/content/drive/MyDrive/kaggle/movies_metadata.csv")
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



Here are some movies you might like (item-based collaborative filtering approach): ['The Great Gatsby', 'Hamlet', 'The Phantom of the Opera', 'Frankenstein', 'A Christmas Carol', 'Eden', 'The Letter', 'Little Women', 'Inside Out', 'Possessed']


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")

    input1 = gr.Textbox(placeholder='first film name...')

    input2 = gr.Textbox(placeholder='second film name...')

    input3 = gr.Textbox(placeholder='third film name...')

    text_output=gr.Textbox(  )


    text_button = gr.Button("Submit")
    text_button.click(get_item_based_recommendations, inputs=[input1,input2,input3], outputs=text_output)
demo.launch()

NameError: ignored

In [ ]:
def get_creative_recommendations(input1,input2,input3):
    # Calculate the average popularity of the input movies
    input_movies=[input1,input2,input3]
    avg_popularity = movies[movies["title"].isin(input_movies)]["popularity"].mean()


    # Check if there are input movies in the dataset
    if pd.isna(avg_popularity):
        print("No input movies found in the dataset.")
        return []

    # Convert avg_popularity to a scalar value
    avg_popularity = avg_popularity.item()

    # Convert popularity column to numeric data type
    movies["popularity"] = pd.to_numeric(movies["popularity"], errors="coerce")

    # Get all movies with popularity above the average
    popular_movies = movies[movies["popularity"] > avg_popularity]

    # Get the unique genres of the input movies
    input_genres = set()
    for movie in input_movies:
        genres = movies[movies["title"] == movie]["genres"].iloc[0]
        input_genres.update(genres)

    # Select movies from popular_movies that have at least one genre different from the input movies
    recommended_movies = []
    for index, movie in popular_movies.iterrows():
        genres = movie["genres"]
        if isinstance(genres, list) and not input_genres.intersection(genres):
            recommended_movies.append(movie["title"])
            if len(recommended_movies) == 10:
                break

    # Return the recommendations list
    return recommended_movies
# Get the recommendations based on the input movies using the item-based collaborative filtering approach
#recommendations = get_item_based_recommendations("The Matrix", "The Godfather", "Titanic")
# Print the recommendations
#print("\nHere are some movies you might like (item-based collaborative filtering approach):")
for movie in recommendations:
    print(movie)

# Get the recommendations based on the input movies using the creative approach
creative_recommendations = get_creative_recommendations("The Matrix", "The Godfather", "Titanic")

# Print the creative recommendations
print("\nHere are some movies you might like (creative approach):")
for movie in creative_recommendations:
    print(movie)


The Great Gatsby
Hamlet
The Phantom of the Opera
Frankenstein
A Christmas Carol
Eden
The Letter
Little Women
Inside Out
Possessed

Here are some movies you might like (creative approach):
DragonHeart
Monsters, Inc.
Harry Potter and the Philosopher's Stone
Spirited Away
Harry Potter and the Chamber of Secrets
Finding Nemo
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
The Mother of Tears
The Hangover


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")

    input1 = gr.Textbox(placeholder='first film name...')

    input2 = gr.Textbox(placeholder='second film name...')

    input3 = gr.Textbox(placeholder='third film name...')

    text_output=gr.Textbox( )


    text_button = gr.Button("Submit")
    text_button.click(get_creative_recommendations, inputs=[input1,input2,input3], outputs=text_output)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

##Recommendation-Specific Methods (Extra Point): it includes collaborative filtering,
content-based filtering, and hybrid methods. Collaborative filtering involves
recommending items to users based on their similarity to other users' preferences.
Content-based filtering, on the other hand, recommends items based on their similarity
to the user's previously liked items. Hybrid methods combine both collaborative and
content-based filtering approaches to provide more accurate and diverse
recommendations. These methods can be implemented as part of your recommender
system to enhance its performance and effectiveness. By leveraging these techniques,
you can provide personalized and relevant recommendations to users based on their
preferences and characteristics. References for these methods are provided in the last
part of the documentation for further reading.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load movie metadata into a pandas DataFrame
movie_data = pd.read_csv('/content/drive/MyDrive/kaggle/movies_metadata.csv', usecols=['title', 'budget', 'revenue', 'vote_count', 'vote_average'])

# Load movie ratings into a pandas DataFrame
ratings_data = pd.read_csv('/content/drive/MyDrive/kaggle/ratings_small.csv')

# Collaborative Filtering
def collaborative_filtering(user_id, num_recommendations=5):
    user_ratings = ratings_data[ratings_data['userId'] == user_id]
    user_ratings = user_ratings.merge(movie_data[['title']], left_on='movieId', right_index=True)
    user_ratings = user_ratings.drop_duplicates(subset=['title'], keep='first')

    # Get the mean ratings for each movie
    movie_ratings_mean = ratings_data.groupby('movieId')['rating'].mean().reset_index()

    # Merge with movie data to get movie titles
    movie_ratings_mean = movie_ratings_mean.merge(movie_data[['title']], left_on='movieId', right_index=True)
    movie_ratings_mean = movie_ratings_mean.drop_duplicates(subset=['title'], keep='first')

    # Sort movies by mean rating in descending order
    top_movies = movie_ratings_mean.sort_values(by='rating', ascending=False)['title'][:num_recommendations]

    return top_movies.tolist()

# Content-Based Filtering
def content_based_filtering(movie_title, num_recommendations=5):
    # Drop duplicates and missing values based on 'title' column
    movie_data_unique = movie_data.drop_duplicates(subset='title', keep='first').dropna(subset=['budget', 'revenue', 'vote_count', 'vote_average'])

    # Calculate similarity scores between input movie and other movies based on selected features
    selected_features = ['budget', 'revenue', 'vote_count', 'vote_average']
    movie_features = movie_data_unique[selected_features]
    similarity_scores = cosine_similarity(movie_features[movie_data_unique['title'] == movie_title], movie_features)
    movie_data_unique['similarity'] = similarity_scores.flatten()

    # Get the most similar movies
    top_movies = movie_data_unique[movie_data_unique['title'] != movie_title].sort_values(by='similarity', ascending=False)['title'][:num_recommendations]

    return top_movies.tolist()


# Hybrid Method
def hybrid_filtering(user_id, movie_title, num_recommendations=5):
    collaborative_movies = collaborative_filtering(user_id)
    content_based_movies = content_based_filtering(movie_title)

    # Merge and deduplicate the recommendations
    all_recommendations = collaborative_movies + content_based_movies
    unique_recommendations = list(set(all_recommendations))[:num_recommendations]

    return unique_recommendations

# Example usage
user_id = 1
movie_title = 'The Dark Knight'
collaborative_recommendations = collaborative_filtering(user_id)
content_based_recommendations = content_based_filtering(movie_title)
hybrid_recommendations = hybrid_filtering(user_id, movie_title)

print("Collaborative Filtering Recommendations:")
print(collaborative_recommendations)

print("\nContent-Based Filtering Recommendations:")
print(content_based_recommendations)

print("\nHybrid Recommendations:")
print(hybrid_recommendations)


Collaborative Filtering Recommendations:
['Death Watch', 'Planet Outlaws', 'Track 29', 'The Trans-Atlantic Mystery', 'The Story of Qiu Ju']

Content-Based Filtering Recommendations:
['O Kadhal Kanmani', 'Bhaag Milkha Bhaag', 'Exodus', 'Next Friday', 'The Berlin File']

Hybrid Recommendations:
['The Story of Qiu Ju', 'O Kadhal Kanmani', 'The Berlin File', 'Planet Outlaws', 'Death Watch']


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")
    title="Movie Recommender System"
    user_id = gr.Textbox(placeholder='user_id')

    #input2 = gr.Textbox(placeholder='second film name...')

    #input3 = gr.Textbox(placeholder='third film name...')

    text_output=gr.Textbox( )


    text_button = gr.Button("Submit")
    text_button.click(collaborative_filtering, inputs=user_id, outputs=text_output)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")
    title="Movie Recommender System"
    movie_title = gr.Textbox(placeholder='movie_title')

    #input2 = gr.Textbox(placeholder='second film name...')

    #input3 = gr.Textbox(placeholder='third film name...')

    text_output=gr.Textbox( )


    text_button = gr.Button("Submit")
    text_button.click(content_based_filtering, inputs= movie_title, outputs=text_output)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")
    title="Movie Recommender System"
    user_id = gr.Textbox(placeholder='user_id')
    movie_title = gr.Textbox(placeholder='movie_title')

    #input2 = gr.Textbox(placeholder='second film name...')

    #input3 = gr.Textbox(placeholder='third film name...')

    text_output=gr.Textbox( )


    text_button = gr.Button("Submit")
    text_button.click(hybrid_filtering, inputs= [ user_id,movie_title], outputs=text_output)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

##extra2

### Extra scores may be granted for more attractive interfaces.


In [ ]:
import tkinter as tk
from tkinter import messagebox

# Function to handle button click event
def recommend_movies():
    user_id = int(input("User ID: "))
    movie_title = input("Movie Title: ")

    # Call the hybrid filtering method to get recommendations
    recommendations = hybrid_filtering(user_id, movie_title)

    # Show recommendations
    print("\nRecommendations:")
    for movie in recommendations:
        print(movie)

# Call the recommend_movies function
recommend_movies()


User ID: 1
Movie Title: Death Watch

Recommendations:
The Story of Qiu Ju
Waterboys
Planet Outlaws
Death Watch
The Bone Snatcher


###To enhance the user experience with recommended movies, you have the option to
utilize the following APIs that provide additional information about movies, including
movie posters. These APIs are compatible with the provided dataset.
○ OMDb

In [ ]:
import requests

def get_movie_info(movie_title):
    # API endpoint and parameters
    url = "http://www.omdbapi.com/"
    params = {
        "apikey": "http://www.omdbapi.com/?i=tt3896198&apikey=dd7e21a6",  # Replace with your actual API key
        "t": movie_title
    }

    # Send GET request to the API
    response = requests.get(url, params=params)
    data = response.json()

    # Extract relevant information from the response
    if response.status_code == 200 and data["Response"] == "True":
        title = data["Title"]
        year = data["Year"]
        director = data["Director"]
        plot = data["Plot"]
        poster_url = data["Poster"]

        # Return the movie information
        return {
            "Title": title,
            #"Year": year,
            #"Director": director,
            #"Plot": plot,
            "Poster": poster_path
        }
    else:
        return None

# Example usage
movie_title = "The Dark Knight"
movie_info = get_movie_info(movie_title)

if movie_info:
    print("Movie Information:")
    print("Title:", movie_info["Title"])
    print("Year:", movie_info["Year"])
    print("Director:", movie_info["Director"])
    print("Plot:", movie_info["Plot"])
    print("Poster:", movie_info["Poster"])
else:
    print("Movie information not found.")


Movie information not found.


###To enhance the user experience with recommended movies, you have the option to
utilize the following APIs that provide additional information about movies, including
movie posters. These APIs are compatible with the provided dataset.
○ TMDB


In [ ]:
import requests
import io

# TMDB API Key
TMDB_API_KEY = "3a5262acd27fd1c561d8d932ea46513c"

# Function to handle button click event
def recommend_movies():
    # Set default values for user_id and movie_title
    user_id = 1
    movie_title = "Next Friday"

    # Call the hybrid filtering method to get recommendations
    recommendations = [
    {"title": "Movie 1", "poster_path": "/path/to/poster1.jpg"},
    {"title": "Movie 2", "poster_path": "/path/to/poster2.jpg"},
    {"title": "Movie 3", "poster_path": "/path/to/poster3.jpg"},
    # ...
]
    # Display recommendations
    print("Recommendations:")
    for movie in recommendations:
        title = movie["title"]
        print(title)

# Call the recommend_movies function
recommend_movies()


Recommendations:
Movie 1
Movie 2
Movie 3


### Evaluate the proposed method’s performance using specific metrics for recommendation
systems.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the Movies Dataset
movies_metadata = pd.read_csv('/content/drive/MyDrive/kaggle/movies_metadata.csv')
keywords = pd.read_csv('/content/drive/MyDrive/kaggle/keywords.csv')
ratings = pd.read_csv('/content/drive/MyDrive/kaggle/ratings_small.csv')

# Preprocess the data
# Example: Select a subset of relevant columns from movies_metadata and ratings
movies = movies_metadata[['title', 'genres', 'release_date', 'popularity']]
ratings = ratings[['userId', 'movieId', 'rating']]

# Split the data into training and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# Build your recommendation model (Collaborative Filtering, Content-Based, etc.)

# Generate recommendations for test data

# Prepare ground truth data for evaluation
actual = test_data['rating'].values

# Replace 'predicted' with the predicted ratings or rankings from your model
predicted = np.random.randint(1, 6, size=len(actual))  # Random predictions as an example

# Calculate evaluation metrics
mse = mean_squared_error(actual, predicted)
rmse = np.sqrt(mse)

# Print evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)


<ipython-input-19-4713be21b986>:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('/content/drive/MyDrive/kaggle/movies_metadata.csv')


Mean Squared Error (MSE): 3.4401654917254136
Root Mean Squared Error (RMSE): 1.8547683121418195
